### Remove bad trials and plot spectrogram
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [24]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PyQt5.QtWidgets.QWidget import width
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, outliers_to_nan,crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [3]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[0]
subject_Tag = 'D103'

In [4]:
raw = raw_from_layout(layout.derivatives['derivatives/cleanbchrm'], subject=subject, desc='cleanbchrm',extension='.edf',preload=False)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-cleanbchrm_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-cleanbchrm_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-01_desc-cleanbchrm_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalD

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 426 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-03_desc-cleanbchrm_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-03_desc-cleanbchrm_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-04_desc-cleanbchrm_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 426 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-04_desc-cleanbchrm_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_task-LexicalDecRepDelay_acq-01_run-04_desc-cleanbchrm_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\cleanbchrm\sub-D0103\ieeg\sub-D0103_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 423 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


In [ ]:
# Epoches: https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.__getitem__
for epoch, t, tag in zip(
                ('Auditory_stim/Repeat/Word/CORRECT','Auditory_stim/Repeat/Nonword/CORRECT',
                 'Auditory_stim/Yes_No/Word/CORRECT', 'Auditory_stim/Yes_No/Nonword/CORRECT',
                 'Delay/Repeat/Word/CORRECT','Delay/Repeat/Nonword/CORRECT',
                 'Delay/Yes_No/Word/CORRECT', 'Delay/Yes_No/Nonword/CORRECT',
                 'Resp/Repeat/Word/CORRECT','Resp/Repeat/Nonword/CORRECT',
                 'Resp/Yes_No/Word/CORRECT', 'Resp/Yes_No/Nonword/CORRECT'),
                ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),
                 (-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),
                 (-0.5, 1),(-0.5, 1),(-0.5, 1),(-0.5, 1)),
                ('Auditory-Rep-Word','Auditory-Rep-Nonword',
                 'Auditory-YN-Word','Auditory-YN-Nonword',
                 'Delay-Rep-Word','Delay-Rep-Nonword',
                 'Delay-YN-Word','Delay-YN-Nonword',
                 'Resp-Rep-Word','Resp-Rep-Nonword',
                 'Resp-YN-Word','Resp-YN-Nonword')
             ):
    
    # Add time point for timefrequency and crop it
    t1 = t[0] - 0.5
    t2 = t[1] + 0.5
    times = (t1, t2)
    trials = trial_ieeg(raw, epoch, times, preload=True)
    # eeg file is big. If try load multiple subjects. 
    # pointing
    
    # remove bad channels
    outliers_to_nan(trials, outliers=10)
    #Learned from Aaron's sentence rep
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    
    # Multitaper wavelet timefreq
    #freq = np.arange(10, 200., 6.)
    #spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
    #                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
    # verbose: extra messages
    # https://mne.tools/dev/generated/mne.verbose.html#mne.verbose
    #spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
    # https://mne.tools/dev/generated/mne.time_frequency.tfr_array_multitaper.html (Notes)
    #crop_pad(spectra, "0.5s")
    freq = np.linspace(0.5, 200, num=80)
    kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                  n_cycles=freq/2, time_bandwidth=4,
                  # n_fft=int(trials.info['sfreq'] * 2.75),
                  decim=20, )
                  # adaptive=True)
    spectra = trials.compute_tfr(method="multitaper",  **kwargs)
    crop_pad(spectra, "0.5s") # cut the first and final 0.5s, change to zero
    
    # crop the baseline
    if tag == 'Auditory-Rep-Word' or 'Auditory-Rep-Nonword' or  'Auditory-YN-Word' or 'Auditory-YN-Nonword': # Have troubles
        base = spectra.copy().crop(-0.5, 0)
            # Average, and do baseline correction
        base = base.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
    spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=True)
    rescale(spectra._data, base._data, mode='ratio', axis=-1)
    
    # Save spectrograms
    #fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    #spectra.info['subject_info']['files'] = tuple(fnames)
    #spectra.info['bads'] = raw.info['bads']
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    if not os.path.exists(os.path.join(save_dir, subject,'multitaper')):
        os.mkdir(os.path.join(save_dir, subject,'multitaper'))
    filename = os.path.join(save_dir, subject,'multitaper',f'{tag}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
    #spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))
    del spectra

In [38]:
# file names
epoch_stim_rep_word =  "Auditory-Rep-Word"
epoch_delay_rep_word=  "Delay-Rep-Word"
epoch_resp_rep_word=  "Resp-Rep-Word"

epoch_stim_rep_nonword =  "Auditory-Rep-Nonword"
epoch_delay_rep_nonword=  "Delay-Rep-Nonword"
epoch_resp_rep_nonword=  "Resp-Rep-Nonword"

epoch_stim_yn_word =  "Auditory-YN-Word"
epoch_delay_yn_word=  "Delay-YN-Word"
epoch_resp_yn_word=  "Resp-YN-Word"

epoch_stim_yn_nonword =  "Auditory-YN-Nonword"
epoch_delay_yn_nonword=  "Delay-YN-Nonword"
epoch_resp_yn_nonword=  "Resp-YN-Nonword"

# filepath
fname_stim_rep_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_stim_rep_word}-tfr.h5')
fname_delay_rep_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_delay_rep_word}-tfr.h5')
fname_resp_rep_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_resp_rep_word}-tfr.h5')

fname_stim_rep_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_stim_rep_nonword}-tfr.h5')
fname_delay_rep_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_delay_rep_nonword}-tfr.h5')
fname_resp_rep_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_resp_rep_nonword}-tfr.h5')

fname_stim_yn_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_stim_yn_word}-tfr.h5')
fname_delay_yn_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_delay_yn_word}-tfr.h5')
fname_resp_yn_word = os.path.join(save_dir, subject,'multitaper',f'{epoch_resp_yn_word}-tfr.h5')

fname_stim_yn_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_stim_yn_nonword}-tfr.h5')
fname_delay_yn_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_delay_yn_nonword}-tfr.h5')
fname_resp_yn_nonword = os.path.join(save_dir, subject,'multitaper',f'{epoch_resp_yn_nonword}-tfr.h5')

# load files
spectra_stim_rep_word = mne.time_frequency.read_tfrs(fname_stim_rep_word)
spectra_delay_rep_word = mne.time_frequency.read_tfrs(fname_delay_rep_word)
spectra_resp_rep_word = mne.time_frequency.read_tfrs(fname_resp_rep_word)

spectra_stim_rep_nonword = mne.time_frequency.read_tfrs(fname_stim_rep_nonword)
spectra_delay_rep_nonword = mne.time_frequency.read_tfrs(fname_delay_rep_nonword)
spectra_resp_rep_nonword = mne.time_frequency.read_tfrs(fname_resp_rep_nonword)

spectra_stim_yn_word = mne.time_frequency.read_tfrs(fname_stim_yn_word)
spectra_delay_yn_word = mne.time_frequency.read_tfrs(fname_delay_yn_word)
spectra_resp_yn_word = mne.time_frequency.read_tfrs(fname_resp_yn_word)

spectra_stim_yn_nonword = mne.time_frequency.read_tfrs(fname_stim_yn_nonword)
spectra_delay_yn_nonword = mne.time_frequency.read_tfrs(fname_delay_yn_nonword)
spectra_resp_yn_nonword = mne.time_frequency.read_tfrs(fname_resp_yn_nonword)

# plot
base_fontsize = 3
fig_width, fig_height = (13.33, 7.5)  # 16:9 ppt scale
dpi = 300
scale_factor = min(fig_width / 10, fig_height / 5)
font_size = base_fontsize * scale_factor
    
for channel_name in spectra_stim_rep_word.ch_names:

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
    plt.rcParams.update({
    'font.size': font_size,            
    'axes.titlesize': font_size * 1.2, 
    'axes.labelsize': font_size,
    'xtick.labelsize': font_size * 0.8, 
    'ytick.labelsize': font_size * 0.8, 
    'legend.fontsize': font_size * 0.8,
    })
    
    gs0 = gridspec.GridSpec(nrows=1, ncols=3, figure=fig)
    
    # Cluster1: Auditory stimuli
    gs00 = gridspec.GridSpecFromSubplotSpec(nrows=3,ncols=4,subplot_spec=gs0[0],width_ratios=[10/31, 10/31, 10/31, 1/32])
    
    ## Plot 1-1: Auditory-Rep-Word
    ax_aud_rep_word = fig.add_subplot(gs00[0,0])
    spectra_stim_rep_word.plot(channel_name, vlim=(0.7, 1.4), show=False, colorbar=False,fmax=200, axes=ax_aud_rep_word,cmap=parula_map,title=channel_name)
    ax_aud_rep_word.set_title('Stim-Rep-Word')
    
    ## Plot 1-2: Auditory-Rep-Nonword
    ax_aud_rep_nonword = fig.add_subplot(gs00[0,1])
    spectra_stim_rep_nonword.plot(channel_name, vlim=(0.7, 1.4), show=False, colorbar=False,fmax=200, axes=ax_aud_rep_nonword,cmap=parula_map,title=channel_name)
    ax_aud_rep_nonword.set_title('Nonword')
    ax_aud_rep_nonword.set_ylabel("")
    ax_aud_rep_nonword.set_yticklabels("")
    
    # Cluster2: Delay
    gs01 = gridspec.GridSpecFromSubplotSpec(nrows=3,ncols=4,subplot_spec=gs0[1],width_ratios=[10/31, 10/31, 10/31, 1/32])

    ## Plot 2-1: Delay-Rep-Word
    ax_del_rep_word = fig.add_subplot(gs01[0,0])
    spectra_stim_rep_word.plot(channel_name, vlim=(0.7, 1.4), show=False, colorbar=False,fmax=200, axes=ax_del_rep_word,cmap=parula_map,title=channel_name)
    ax_del_rep_word.set_title('Delay-Rep-Word')
    ax_del_rep_word.set_ylabel("")
    ax_del_rep_word.set_yticklabels("")
    
    ## Plot 2-2: Delay-Rep-Nonword
    ax_del_rep_nonword = fig.add_subplot(gs01[0,1])
    spectra_delay_rep_nonword.plot(channel_name, vlim=(0.7, 1.4), show=False, colorbar=False,fmax=200, axes=ax_del_rep_nonword,cmap=parula_map,title=channel_name)
    ax_del_rep_nonword.set_title('Nonword')
    ax_del_rep_nonword.set_ylabel("")
    ax_del_rep_nonword.set_yticklabels("")
    
    # Cluster3: Resp
    gs02 = gridspec.GridSpecFromSubplotSpec(nrows=3,ncols=4,subplot_spec=gs0[2],width_ratios=[10/31, 10/31, 10/31, 1/32])
    
    ## Plot 3-1: Resp-Rep-Word
    ax_res_rep_word = fig.add_subplot(gs02[0,0])
    spectra_stim_rep_word.plot(channel_name, vlim=(0.7, 1.4), show=False, colorbar=False,fmax=200, axes=ax_res_rep_word,cmap=parula_map,title=channel_name)
    ax_res_rep_word.set_title('Resp-Rep-Word')
    ax_res_rep_word.set_ylabel("")
    ax_res_rep_word.set_yticklabels("")
    
    # Colorbar
    ax_colorbar_rep = fig.add_subplot(gs02[0,3])
    fig.colorbar(ax_res_rep_word.images[-1], cax=ax_colorbar_rep).ax.set_yscale("linear")
        
    plt.show()
#chan_grid(spectra, size = (20,10),vlim=(0.7, 1.4), fmax=200, show=True, cmap=parula_map,)# yscale="log")


Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Auditory-Rep-Word-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Delay-Rep-Word-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Resp-Rep-Word-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Auditory-Rep-Nonword-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Delay-Rep-Nonword-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Resp-Rep-Nonword-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings\Baishen\ieeg_results\lexical_delay\D0103\multitaper\Auditory-YN-Word-tfr.h5 ...
Reading C:\Users\bl314\Box\CoganLab\IndividualMeetings

KeyboardInterrupt: 